In [1]:
import pandas as pd
import numpy as np
import time
import re
import copy
import random
import os
from tqdm import tqdm_notebook
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys


In [196]:
# 프록시 주소를 중간중간 변경하여 로봇에 탐지되는 것을 회피하기 위한 코드
# 시스템 프록시 설정 읽기 (예: Windows 환경 변수 사용)

proxy = os.environ.get('http_proxy') or os.environ.get('https_proxy')

options = webdriver.ChromeOptions()
options.add_argument(f'--proxy-server={proxy}')

driver = webdriver.Chrome(options=options)


In [2]:
df = pd.read_excel('data/imdb_combine_240611.xlsx', index_col=0)
df.head()

,title,english_title,year,total_season_num,runtime,genre,age_rating,production_country,total_rate,total_count,...,season_8,season_9,season_10,season_11,season_12,season_13,season_14,season_15,season_16,season_17
0,Ojingeo Geim,Squid Game,2021,1,55,"액션, 드라마, 스릴러",18,대한민국,8.0,549000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12인의 심판자,NaN,2019,2,54,"드라마, 스릴러",18,벨기에,7.4,6500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,희생자 게임,The Victims' Game,2020,1,60,"드라마, 범죄, 스릴러",18,대만,7.3,1900.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,스위트 투스: 사슴뿔을 가진 소년,Sweet Tooth,2021,2,52,"SF, 드라마, 판타지, 액션",15,미국,7.7,68000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Sweet Home,Sweet Home,2020,2,60,"스릴러, 드라마, 판타지, 공포, SF, 범죄",18,대한민국,7.3,33000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
countries_dict = {
    '대한민국': 'Korea',
    '벨기에': 'Belgium',
    '대만': 'Taiwan',
    '미국': 'US',
    '덴마크': 'Denmark',
    '독일': 'Germany',
    '일본': 'Japan',
    '태국': 'Thailand',
    '캐나다': 'Canada',
    '그리스': 'Greece',
    '영국': 'United Kingdom',
    'China': 'China',
    '스페인': 'Spain',
    '미국, 영국': 'US, United Kingdom',
    '아이슬란드, 덴마크, 핀란드, 스웨덴, 노르웨이, 독일, 프랑스, 영국': 'Iceland, Denmark, Finland, Sweden, Norway, Germany, France, United Kingdom',
    '미국, 프랑스': 'US, France',
    '호주': 'Australia',
    '스웨덴': 'Sweden',
    '이탈리아': 'Italy',
    '케냐': 'Kenya',
    'WEBAPP_COUNTRY_VN': 'Vietnam',
    '인도': 'India',
    '호주, 미국': 'Australia, US',
    '스위스': 'Switzerland',
    '필리핀 제도': 'Philippines',
    '영국, 미국': 'United Kingdom, US',
    '남아프리카공화국': 'South Africa',
    '폴란드': 'Poland',
    '브라질': 'Brazil',
    '미국, 캐나다': 'US, Canada',
    '콜롬비아': 'Colombia',
    '터키': 'Turkey',
    '룩셈부르크': 'Luxembourg',
    '핀란드': 'Finland',
    '이스라엘': 'Israel',
    '프랑스': 'France',
    '멕시코': 'Mexico',
    '벨기에, 네덜란드': 'Belgium, Netherlands',
    '덴마크, 노르웨이': 'Denmark, Norway',
    '노르웨이': 'Norway',
    '러시아': 'Russia',
    '미국, 일본': 'US, Japan',
    '영국, 프랑스': 'United Kingdom, France',
    '요르단': 'Jordan',
    '프랑스, 미국': 'France, US',
    '캐나다, 영국': 'Canada, United Kingdom',
    '태국, 미국': 'Thailand, US',
    '인도네시아': 'Indonesia',
    '사우디아라비아': 'Saudi Arabia',
    '영국, 인도': 'United Kingdom, India',
    'Arjantin': 'Argentina',
    '아이슬란드': 'Iceland',
    '오스트리아': 'Austria',
    '포르투갈': 'Portugal',
    '프랑스, 독일, 스페인, 영국': 'France, Germany, Spain, United Kingdom',
    '아일랜드, 영국': 'Ireland, United Kingdom',
    '캐나다, 프랑스, 독일': 'Canada, France, Germany',
    '이스라엘, 노르웨이': 'Israel, Norway',
    '대한민국, 미국': 'South Korea, US',
    '미국, 대한민국': 'US, South Korea',
    '네덜란드': 'Netherlands',
    '멕시코, 스페인': 'Mexico, Spain',
    '홍콩': 'Hong Kong',
    '칠레': 'Chile',
    '뉴질랜드, 호주': 'New Zealand, Australia',
    '나이지리아': 'Nigeria',
    '이집트': 'Egypt',
    '일본, 미국': 'Japan, US',
    '미국, 멕시코': 'US, Mexico',
    '쿠웨이트': 'Kuwait',
    '나이지리아, 미국': 'Nigeria, US',
    '말레이시아, 대만': 'Malaysia, Taiwan',
    '사우디아라비아, 인도': 'Saudi Arabia, India',
    '벨기에, 룩셈부르크': 'Belgium, Luxembourg',
    '사우디아라비아, 튀니지, 이집트, 모로코, 레바논': 'Saudi Arabia, Tunisia, Egypt, Morocco, Lebanon',
    '독일, 오스트리아': 'Germany, Austria',
    '체코 공화국, 독일': 'Czech Republic, Germany',
    '영국, 스페인': 'United Kingdom, Spain',
    '스페인, 브라질': 'Spain, Brazil',
    '레바논': 'Lebanon'
}

In [197]:
driver = webdriver.Chrome()
driver.get('https://www.justwatch.com/us/')

In [6]:
df.loc[0]['add_english_title'], df.loc[0]['year']

('Squid Game', 2021, '대한민국')

In [10]:
title = df.loc[0]['add_english_title']
year = df.loc[0]['year']

In [53]:
# 검색창에 검색어 입력
driver.find_element(By.CSS_SELECTOR, 'div.navbar__search > div > ion-searchbar > div > input').send_keys(f'{title}')

In [13]:
# 검색 실시
driver.find_element(By.CSS_SELECTOR, 'div.navbar__search > div > ion-searchbar > div > input').send_keys(Keys.ENTER)

In [ ]:
driver.find_element(By.CSS_SELECTOR, 'div.navbar__search > div > ion-searchbar > div > input').send_keys(Keys.ENTER)

In [54]:
driver.find_element(By.CSS_SELECTOR, 'div.navbar__search > div > ion-searchbar > div > input').click()
driver.find_element(By.CSS_SELECTOR, 'ion-searchbar > div > button').click()

In [16]:
# 첫번째 검색 결과물의 제목
driver.find_element(By.CLASS_NAME, 'header-title').text

'Squid Game'

In [30]:
# 첫번째 검색 결과물 클릭
driver.find_element(By.CLASS_NAME, 'header-title').click()

In [17]:
# 연령 정보 추출
age = driver.find_elements(By.CLASS_NAME, 'detail-infos')[4].text
age

'AGE RATING\nTV-14'

In [93]:
df.loc[2,'runtime'].replace('시간', 'h').replace('분', 'min')

'1h 0min'

In [94]:
age.split('\n')[0], age.split('\n')[1]

('RUNTIME', '1h 0min')

In [35]:
driver.find_element(By.CLASS_NAME, 'header-year').text[1:5]

'2021'

In [6]:
# 데이터 프레임에 컬럼 준비
df['justwatch_us_age'] = None

In [37]:
df.head()

,title,original_title,year,season_episode,runtime,genre,age_rating,Production country,IMDb_title,IMDb_URL,add_original_title,add_english_title,justwatch_us_age
0,Ojingeo Geim,Squid Game,2021,2,55min,"액션, 드라마, 스릴러",19,대한민국,Squid Game (TV Series 2021,https://www.imdb.com/title/tt10919420/,오리지날 타이틀 없음,Squid Game,TV-MA
1,12인의 심판자,The Twelve,2019,2,54min,"드라마, 스릴러",NaN,벨기에,The Twelve (TV Mini Series 2022,https://www.imdb.com/title/tt13589004/,오리지날 타이틀 없음,The Twelve,검색 결과 부정확
2,희생자 게임,誰是被害者,2020,2,1시간 0분,"드라마, 범죄, 스릴러",19,대만,The Victims' Game (TV Series 2020,https://www.imdb.com/title/tt12079212/,誰是被害者,The Victims' Game,TV-MA
3,스위트 투스: 사슴뿔을 가진 소년,Sweet Tooth,2021,3,52min,"SF, 드라마, 판타지, 액션",15,미국,Sweet Tooth (TV Series 2021,https://www.imdb.com/title/tt12809988/,오리지날 타이틀 없음,Sweet Tooth,런타임 다름
4,Sweet Home,NaN,2020,3,1시간 0분,"스릴러, 드라마, 판타지, 공포, SF, 범죄",NaN,대한민국,Sweet Home (TV Series 2020,https://www.imdb.com/title/tt11612120/,오리지날 타이틀 없음,Sweet Home,TV-MA


In [ ]:
# justwatch에서 직접 검색하여 크롤링 하기
# 검색 결과 첫번째만 확인

for idx, row in tqdm_notebook(df.iterrows()):
    # 데이터 프레임의 영문 제목과 작품의 공개 연도를 변수에 할당
    search_title, search_year = row['add_english_title'], row['year']
    time.sleep(2)

    # 검색창에 검색어 입력
    driver.find_element(By.CSS_SELECTOR, 'div.navbar__search > div > ion-searchbar > div > input').send_keys(f'{search_title}')
    # 검색 실시
    driver.find_element(By.CSS_SELECTOR, 'div.navbar__search > div > ion-searchbar > div > input').send_keys(Keys.ENTER)
    time.sleep(2)

    # 첫번째 검색 결과물의 제목
    title = driver.find_element(By.CLASS_NAME, 'header-title').text
    # 첫번째 검색 결과물의 연도
    year = int(driver.find_element(By.CLASS_NAME, 'header-year').text[1:5])

    if search_title == title and search_year == year:
        # 첫번째 검색 결과물 클릭
        driver.find_element(By.CLASS_NAME, 'header-title').click()
    else:
        df.loc[idx, 'justwatch_us_age'] = '검색 결과 부정확'
        driver.find_element(By.CSS_SELECTOR, 'div.navbar__search > div > ion-searchbar > div > input').click()
        driver.find_element(By.CSS_SELECTOR, 'ion-searchbar > div > button').click()
        continue
    
    time.sleep(2)

    # 연령 정보 추출
    drama_info1 = driver.find_elements(By.CLASS_NAME, 'detail-infos')
    age_info_tf = False
    for elements1 in drama_info1:
        if len(elements1.text.split('\n')) > 1:
            info_name1, runtime_info = elements1.text.split('\n')[0], elements1.text.split('\n')[1]
            if info_name1 == 'RUNTIME' and runtime_info == row['runtime'].replace('시간', 'h').replace('분', 'min'):
                age_info_tf = True
                break
            df.loc[idx, 'justwatch_us_age'] = '런타임 다름'

    if age_info_tf:
        drama_info2 = driver.find_elements(By.CLASS_NAME, 'detail-infos')
        for elements2 in drama_info2:
            if len(elements2.text.split('\n')) > 1:
                info_name2, age_info = elements2.text.split('\n')[0], elements2.text.split('\n')[1]
                if info_name2 == 'AGE RATING':
                    df.loc[idx, 'justwatch_us_age'] = age_info
                    break
                elif info_name2 != 'AGE RATING':
                    df.loc[idx, 'justwatch_us_age'] = '연령 정보 없음'
                    continue

    try:
        # 버튼 찾기 (예: id가 'submit_button'인 버튼)
        button = driver.find_element(By.CSS_SELECTOR, "ion-header > ion-toolbar > ion-buttons > ion-button").click()
    except:
        pass

    driver.find_element(By.CSS_SELECTOR, 'div.navbar__search > div > ion-searchbar > div > input').send_keys(Keys.BACKSPACE * len(search_title))

In [153]:
driver.find_elements(By.CLASS_NAME, 'title-list-row__column-header')[2].get_attribute('href')

'https://www.justwatch.com/us/tv-show/the-kingdom-2021-0'

In [184]:
action = ActionChains(driver)

In [ ]:
# justwatch에서 직접 검색하여 크롤링 하기
# 검색 결과 전체 리스트 확인

for idx, row in tqdm_notebook(df[df['justwatch_us_age'] == '검색 결과 부정확'].iterrows()):
    # 데이터 프레임의 영문 제목과 작품의 공개 연도를 변수에 할당
    search_title, search_year = row['add_english_title'], row['year']
    time.sleep(2)

    # 검색창에 검색어 입력
    driver.find_element(By.CSS_SELECTOR, 'div.navbar__search > div > ion-searchbar > div > input').send_keys(f'{search_title}')
    # 검색 실시
    driver.find_element(By.CSS_SELECTOR, 'div.navbar__search > div > ion-searchbar > div > input').send_keys(Keys.ENTER)
    time.sleep(3)

    # 첫번째 검색 결과물의 제목
    title = driver.find_element(By.CLASS_NAME, 'header-title').text
    # 첫번째 검색 결과물의 연도
    year = int(driver.find_element(By.CLASS_NAME, 'header-year').text[1:5])
    t = 0
    for item in driver.find_elements(By.CLASS_NAME, 'title-list-row__column-header'):
        if len(item.text.split('(')) < 2:
            t += 1
            continue
        else:
            title = item.text.split('(')[0]
            year = int(item.text.split('(')[1][:-1])
            if search_title == title and search_year == year:
                driver.get(item.get_attribute('href'))
                break
    if search_title != title or search_year != year:
        df.loc[idx, 'justwatch_us_age'] = '검색 결과 부정확'
        driver.find_element(By.CSS_SELECTOR, 'div.navbar__search > div > ion-searchbar > div > input').click()
        driver.find_element(By.CSS_SELECTOR, 'ion-searchbar > div > button').click()
        continue
    
    time.sleep(2)

    # 연령 정보 추출
    drama_info1 = driver.find_elements(By.CLASS_NAME, 'detail-infos')
    age_info_tf = False
    for elements1 in drama_info1:
        if len(elements1.text.split('\n')) > 1:
            info_name1, runtime_info = elements1.text.split('\n')[0], elements1.text.split('\n')[1]
            if info_name1 == 'RUNTIME' and runtime_info == row['runtime'].replace('시간', 'h').replace('분', 'min'):
                age_info_tf = True
                break
            df.loc[idx, 'justwatch_us_age'] = '런타임 다름'

    if age_info_tf:
        drama_info2 = driver.find_elements(By.CLASS_NAME, 'detail-infos')
        for elements2 in drama_info2:
            if len(elements2.text.split('\n')) > 1:
                info_name2, age_info = elements2.text.split('\n')[0], elements2.text.split('\n')[1]
                if info_name2 == 'AGE RATING':
                    df.loc[idx, 'justwatch_us_age'] = age_info
                    break
                elif info_name2 != 'AGE RATING':
                    df.loc[idx, 'justwatch_us_age'] = '연령 정보 없음'
                    continue

    try:
        # 버튼 찾기 (예: id가 'submit_button'인 버튼)
        button = driver.find_element(By.CSS_SELECTOR, "ion-header > ion-toolbar > ion-buttons > ion-button").click()
    except:
        pass

    driver.find_element(By.CSS_SELECTOR, 'div.navbar__search > div > ion-searchbar > div > input').send_keys(Keys.BACKSPACE * len(search_title))

In [203]:
df.to_excel('justwatch_us_age.xlsx')

In [209]:
df.to_csv('justwatch_us_age.CSV')